In [1]:
from bitgauss import BitMatrix

`bitgauss` is a library for doing linear algebra over the two-element field $\mathbb F_2$, i.e. the field whose elements are $\{0, 1\}$, addition is XOR and multiplication is AND.

All features are accessible via `BitMatrix` class, which contains a 2D matrix of bits, bitpacked into 64-bit chunks and stored in row-major order. The class provides methods for creating matrices, and performing Gaussian elimination.

There are several ways to construct a `BitMatrix` using Python bindings.

In [2]:
# BitMatrix.from_int_list creates a BitMatrix from a list of lists of integers (0 or 1)
m = BitMatrix.from_int_list([[1, 0, 1, 0],
                             [0, 1, 0, 1],
                             [1, 1, 0, 0],
                             [0, 0, 1, 1]])
print(m)

# ...whereas BitMatrix.from_list creates a BitMatrix from a list of booleans
m1 = BitMatrix.from_list([[True, False, True, False],
                          [False, True, False, True],
                          [True, True, False, False],
                          [False, False, True, True]])
print(m1)

m == m1

 1  0  1  0 
 0  1  0  1 
 1  1  0  0 
 0  0  1  1 

 1  0  1  0 
 0  1  0  1 
 1  1  0  0 
 0  0  1  1 



True

In [3]:
# BitMatrix.zeros creates a zero matrix of given dimensions
m = BitMatrix.zeros(3, 4)

print(m)

# BitMatrix.identity creates an identity matrix of given size
m1 = BitMatrix.identity(4)

print(m1)

 0  0  0  0 
 0  0  0  0 
 0  0  0  0 

 1  0  0  0 
 0  1  0  0 
 0  0  1  0 
 0  0  0  1 



In [4]:
# BitMatrix.build takes the rows, columns, and a boolean-valued function to build a matrix
m = BitMatrix.build(10, 15, lambda i, j: (i + j) % 2 == 0)
print(m)

 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1 
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0 
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1 
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0 
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1 
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0 
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1 
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0 
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1 
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0 



In [5]:
# BitMatrix.random creates a random matrix of given dimensions
m = BitMatrix.random(4, 5)

print(m)

# It can also take a seed to reproduce the same random matrix
m1 = BitMatrix.random(4, 5, seed=42)
m2 = BitMatrix.random(4, 5, seed=42)

assert(m1 == m2)

# Random invertible matrices can be created with BitMatrix.random_invertible
m3 = BitMatrix.random_invertible(5, seed=42)

print(m3)
print("Rank = ", m3.rank())

 0  0  0  0  0 
 1  0  0  1  0 
 0  0  1  1  0 
 0  0  0  0  0 

 0  0  1  0  1 
 1  1  1  1  0 
 1  0  1  1  1 
 0  0  0  0  1 
 0  0  0  1  0 

Rank =  5


Matrix multiplication is implemented using the `@` operator (or `*`, which is used as an alias for `@`).

In [6]:
m = BitMatrix.random(4, 5, seed=42)
m1 = BitMatrix.random(5, 3, seed=42)

print(m @ m1)

 0  0  1 
 1  1  1 
 0  0  0 
 1  0  0 



Gaussian elimination can be performed using the `gauss` method, which transforms the matrix into row echelon form. The optional parameter `full` can be set to `True` to perform full Gaussian elimination, which reduces the matrix to reduced row echelon form.

In [7]:
m = BitMatrix.random(5, 15, seed=42)
m1 = m.copy()
m1.gauss()

m2 = m.copy()
m2.gauss(full=True)

print(m1)
print(m2)

 1  1  0  1  0  0  0  0  0  1  1  1  0  1  1 
 0  1  0  1  0  0  0  1  1  0  0  1  1  1  1 
 0  0  1  0  1  0  1  1  1  0  0  1  0  1  1 
 0  0  0  1  1  0  0  1  0  0  0  0  0  0  1 
 0  0  0  0  0  0  1  0  0  1  0  1  0  1  1 

 1  0  0  0  0  0  0  1  1  1  1  0  1  0  0 
 0  1  0  0  1  0  0  0  1  0  0  1  1  1  0 
 0  0  1  0  1  0  0  1  1  1  0  0  0  0  0 
 0  0  0  1  1  0  0  1  0  0  0  0  0  0  1 
 0  0  0  0  0  0  1  0  0  1  0  1  0  1  1 



Several other methods use Gaussian elimination under the hood, such as `inverse` and `rank`.

In [8]:
m = BitMatrix.from_int_list([[1, 0, 1, 0],
                             [0, 1, 0, 1],
                             [0, 1, 0, 1],
                             [0, 0, 1, 1]])
print("Rank m =", m.rank())

m1 = BitMatrix.random_invertible(4, seed=42)
print("Rank m1 =", m1.rank())

print()
print(m1.inverse())
assert(m1 @ m1.inverse() == BitMatrix.identity(4))

Rank m = 3
Rank m1 = 4

 0  1  1  0 
 1  1  0  0 
 0  1  0  0 
 0  1  0  1 



### Performance

We'll do a couple of benchmarks to compare performance to a pure Python implementation. This is meant to give a rough idea, not be a comprehensive benchmark vs. the state of the art.

For comparison, we will use a pure Python implementation of Gaussian elimination over $\mathbb F_2$, which came from [this GitHub repo](https://github.com/akissinger/f2linalg).

In [9]:
from f2linalg import Mat2 # pure Python implementation of F2-matrices, for comparison

In [10]:
# Mat2 is constructed from a list of lists of integers, so we can use
# BitMatrix.to_int_list to turn a BitMatrix into a Mat2

m = BitMatrix.random(4, 5, seed=42)
m1 = Mat2(m.to_int_list())
print(m)
print(m1)

 1  1  0  1  0 
 0  1  0  1  0 
 1  1  1  1  1 
 1  0  1  1  0 

 1  1  0  1  0 
 0  1  0  1  0 
 1  1  1  1  1 
 1  0  1  1  0 


We should see a modest improvement for small matrices.

_Note we make a copy each time, otherwise we would do all the work in the first iteration. The overhead of copying is pretty small compared to `gauss`, but if we wanted to more careful, we could measure that part separately._

In [11]:
m = BitMatrix.random(10, 20, seed=42)
m1 = Mat2(m.to_int_list())

%timeit m.copy().gauss()
%timeit m1.copy().gauss()

7.19 μs ± 121 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
45.4 μs ± 1.18 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


...which starts to be significant once we have hundreds of rows/columns:

In [12]:
m = BitMatrix.random(100, 200, seed=42)
m1 = Mat2(m.to_int_list())

%timeit m.copy().gauss()
%timeit m1.copy().gauss()

328 μs ± 19.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
17.9 ms ± 481 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


...and is about 100X faster once we have thousands of rows/columns:

In [13]:
m = BitMatrix.random(1000, 2000, seed=42)
m1 = Mat2(m.to_int_list())

%timeit m.copy().gauss()
%timeit m1.copy().gauss()

82.2 ms ± 1.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
10.1 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
